# Real time Emotion Detection on Video - Overlay

In [1]:
import numpy as np
from keras.models import model_from_json, load_model
import face_recognition
import cv2

from functions import *


/anaconda3/envs/kerasv6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
gender_model = load_model('./Trained Model/trained_GenderNet.hdf5')
gender_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 64)        640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 46, 46, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 46, 46, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 44, 44, 64)        36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 44, 44, 64)        256       
_________________________________________________________________
activation_2 (Activation)    (None, 44, 44, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 22, 64)        0         
__________

In [3]:
# load json and create model
json_file = open('./Trained Model/face_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("./Trained Model/face_model.h5")
print("Loaded model from disk")

model = loaded_model
model.summary()

Loaded model from disk
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 64)        640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 46, 46, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 46, 46, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 44, 44, 64)        36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 44, 44, 64)        256       
_________________________________________________________________
activation_2 (Activation)    (None, 44, 44, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 22, 64)      

## Dataset Explanation

The faces have been automatically registered so that the face is more or less centered and occupies about the same amount of space in each image. The task is to categorize each face based on the emotion shown in the facial expression in to one of seven categories (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral).

The output is a 7 dim sofmax

train.csv contains two columns, "emotion" and "pixels". The "emotion" column contains a numeric code ranging from 0 to 6, inclusive, for the emotion that is present in the image. The "pixels" column contains a string surrounded in quotes for each image. The contents of this string a space-separated pixel values in row major order. test.csv contains only the "pixels" column and your task is to predict the emotion column.

The training set consists of 28,709 examples. The public test set used for the leaderboard consists of 3,589 examples. The final test set, which was used to determine the winner of the competition, consists of another 3,589 examples.




## Functions

In [4]:
def decode(net_output):
    '''EmotionNet output decoder.
    
    '''
    if (np.argmax(net_output) == 0):
        prediction = 'Angry'
    elif (np.argmax(net_output) == 1):
        prediction = 'Disgust'
    elif (np.argmax(net_output) == 2):
        prediction = 'Fear'
    elif (np.argmax(net_output) == 3):
        prediction = 'Happy'
    elif (np.argmax(net_output) == 4):
        prediction = 'Sad'
    elif (np.argmax(net_output) == 5):
        prediction = 'Surprise'
    else:
        prediction = 'Neutral'
        
    return prediction


def decode2(net_output, threshold = 0.3):
    '''EmotionNet output decoder.
    
    '''
    # Check if probability is higher than threshold
    if np.max(net_output) >= threshold:
        probability = str(np.max(net_output))
        if (np.argmax(net_output) == 0):
            prediction = 'Angry'
        elif (np.argmax(net_output) == 1):
            prediction = 'Disgust'
        elif (np.argmax(net_output) == 2):
            prediction = 'Fear'
        elif (np.argmax(net_output) == 3):
            prediction = 'Happy'
        elif (np.argmax(net_output) == 4):
            prediction = 'Sad'
        elif (np.argmax(net_output) == 5):
            prediction = 'Surprise'
        else:
            prediction = 'Neutral'
    else:
        # Do not label the box if you are not sure
        prediction = ''
        probability = ''
    return prediction + ' ' + probability[:5]
       
       
def pre_processing(color_image_rectangle):
    '''Transform input image to EmotionNet input size.
    
    '''
    # Convert image to B&W
    gray_img = cv2.cvtColor(color_image_rectangle,cv2.COLOR_BGR2GRAY)

    # Resize image
    gray_img = cv2.resize(gray_img,(48,48), interpolation = cv2.INTER_CUBIC)

    # Reshape and add mini-batch dimension
    gray_img = np.reshape(gray_img,(48,48,1))
    gray_img = np.expand_dims(gray_img, axis=0)
                      
    return gray_img

def gender_decoder(net_output, threshold=0.65):
    '''GenderNet label decoder -----------------------

    0: male
    1: female
    threshold: Probability minimum threshold to assign a label.
    '''
    if np.max(net_output) > threshold:
        probability = str(np.max(net_output))
        if np.argmax(net_output) == 0:
            prediction = 'Male'
        else:
            prediction = 'Female'
    else:
        prediction = ''
        probability = ''

    return prediction + ' ' + probability[:5]
                    

In [5]:

# Load video
vidcap = cv2.VideoCapture('./Videos/sad_affleck.mp4')

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

while True:
    # Grab a single frame of video
    #ret, frame = video_capture.read()
    success , frame = vidcap.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    
    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]  # size (180, 320, 3)
    
    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame,
                                                        number_of_times_to_upsample=1)
        #face_locations = face_recognition.face_locations(rgb_small_frame, 
        #                                                 model='CNN',
        #                                                 number_of_times_to_upsample=1)
        
        face_emotions = []
        for face in face_locations:
            
            # Get face coordinates
            top, right, bottom, left = face

            # Crop image 
            face_image = rgb_small_frame[top:bottom, left:right]
            
            # Pre-process image
            gray_img = pre_processing(face_image)
            
            # Call model.predict() method
            net_output = model.predict(gray_img)
            net2_output = gender_model.predict(gray_img)

            #print(net_output)
            
            # Decode output
            prediction = decode2(net_output,0.30)
            gender_prediction = gender_decoder(net2_output)
            
            prediction = prediction + '\n' + gender_prediction
            
            # Add emotion to rectangle name
            face_emotions.append(prediction)

    process_this_frame = not process_this_frame

    # Display the results
    for (top, right, bottom, left), emotion in zip(face_locations, face_emotions):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4
        
        overlay = frame.copy()
    
        # Draw a box around the face
#       cv2.rectangle(overlay, (left, top), (right, bottom), (0, 0, 255), 2) # Red
#        cv2.rectangle(overlay, (left, top), (right, bottom), (139, 0, 0), 2) # Blue
        cv2.rectangle(overlay, (left, top), (right, bottom), (180, 0, 0), 2) #Blue

        font = cv2.FONT_HERSHEY_SIMPLEX
        y0, dy = 0, 30
        for i, line in enumerate(emotion.split('\n')):
            y = y0 + i*dy
            cv2.putText(overlay, line, (left + 6, bottom - 6 + y), font, 1.0, (255, 255, 255), 1)
            #cv2.putText(overlay, line, (50, y ), cv2.FONT_HERSHEY_SIMPLEX, 1, 2)


        # Draw a label with a name below the face
        #cv2.rectangle(overlay, (left, bottom - 35), (right, bottom + 50), (0, 0, 255), cv2.FILLED)
        #font = cv2.FONT_HERSHEY_DUPLEX
 
        ############## REAL ONE
        ##font = cv2.FONT_HERSHEY_SIMPLEX
        ##cv2.putText(overlay, emotion, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
        ######################
    
        # cv2.putText(frame, emotion, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        #cv2.circle(overlay, (133, 132), 12, (0, 255, 0), -1)
        opacity = 0.5
        cv2.addWeighted(overlay, opacity, frame, 1 - opacity, 0, frame)
 
    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
#video_capture.release()
cv2.destroyAllWindows()

error: /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/resize.cpp:4044: error: (-215) ssize.width > 0 && ssize.height > 0 in function resize
